## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datetime import datetime
import glob
import json
import pandas as pd
from pathlib import Path

In [4]:
def extract_all_json():

    path = '/Users/mike/IBM-Data-Engineering-Specialization-Coursera/IBM-Data-Engineering-Specialization-Coursera/ETL_Engineer'
    all_json = Path(path).glob('*.json')
    dfs = []

    for jsonfile in all_json:

        df = pd.read_json(jsonfile, lines=False)
        dfs.append(df)

    out_json = pd.concat(dfs, ignore_index=True)

    return out_json

In [5]:
extracted_json = extract_all_json()
extracted_json

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


In [6]:
exchange_rates = pd.read_csv('exchange_rates.csv', index_col=0)
exchange_rates

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083
CNY,6.483139
CZK,21.510117
DKK,6.119757
EUR,0.822504
GBP,0.732398


In [7]:
exchange_rates.at['GBP','Rates']

0.7323984208000001

In [8]:
def transform(data, exchange_rate):

    data['Market Cap (GBP$ Billion)'] = round(data['Market Cap (US$ Billion)'] * exchange_rate, 3)
    data.drop(columns = ['Market Cap (US$ Billion)'], inplace = True)

    return data

In [9]:
transformed_data = transform(extracted_json,exchange_rates.at['GBP','Rates'])
transformed_data

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519
...,...,...
100,Ping An Bank,27.826
101,Standard Chartered,27.332
102,United Overseas Bank,25.728
103,QNB Group,24.579


In [10]:
tmpfile = "temp.tmp"
logfile = "logfile.txt"
targetfile = "transformed_data.csv"

In [11]:
def load(data, targetfile):

    data.to_csv(targetfile)

In [12]:
def log(message):

    timestamp_format = "%Y-%h-%d-%H:%M:%S"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [13]:
log("ETL Job Started")
log("Extract phase Started")
extracted_data = extract_all_json()
log("Extract phase Ended")
extracted_data
log("Transform phase Started")
transformed_data = transform(extracted_data,exchange_rates.at['GBP','Rates'])
log("Transform phase Ended")
transformed_data
log("Load phase Started")
load(transformed_data, targetfile='bank_market_cap_gbp.csv')
log("Load phase Ended")
log("ETL Job Ended")